In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
import re

In [2]:
def get_soup(url):
    t = 1
    #If the url is either a product page or a page within category (i.e. a slow-loading page), wait longer
    if url.startswith("https://www.amazon.com/dp/") or url.startswith("https://www.amazon.com/s?i"):
        time_wait = 20
    #else it's a review page and those load faster
    else:
        time_wait = 5
    while True:
        if t == 3:
            #throw error if a page url doesn't work after 3 times
            raise Exception(f"too many timeouts (3) for url: {url}")
        #via docker
        r = requests.get("http://localhost:8050/render.html", params={"url": url,'wait': time_wait, 'images': 0})
        soup = BeautifulSoup(r.text, "html.parser")
        #error 504 is a globaltimeout error, meaning splash couldn't keep up
        if soup.text.startswith('{"error": 504'):
            time.sleep(31)
            print(f"time sleep try {t}")
            time_wait += 10
            t += 1
        elif soup.title.text == 'Amazon.com':
            raise Exception("blocked")
        else:
            print(f"page successfully loaded in time {time_wait}: {url}")
            break       
    return soup

In [9]:
r = requests.get("http://localhost:8050/render.html", params={"url": "https://www.amazon.com/dp/B07YMNVXVV",'wait': 4, 'viewport': '320x480', 'images': 0})
soup = BeautifulSoup(r.text, "html.parser")
soup.title.text

'Amazon.com'

In [5]:
#{"error": 504, "type": "GlobalTimeoutError", "description": "Timeout exceeded rendering page", "info": {"remaining": -3.588108777999878, "timeout": 30}}

In [20]:
#with open(f"../../gen/input/amazon_500+.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["asin_url", "brand", "model", "link", "product_title",
             "name", "rating", "review_title", "country", "date",
             "specs", "first_spec", "review", "helpful", "picture",
            "verified"])

In [3]:
product_infos = pd.read_excel("../../data/links_over_500_reviews.xlsx")
product_infos

,link,brand,model,title
0,https://www.amazon.com/Apple-iPhone-XR-Fully-U...,apple,Iphone XR,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)"
1,https://www.amazon.com/Apple-iPhone-XR-Fully-U...,apple,Iphone XR,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)"
2,https://www.amazon.com/Apple-iPhone-XR-Fully-U...,apple,Iphone XR,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)"
3,https://www.amazon.com/Apple-iPhone-XR-Fully-U...,apple,Iphone XR,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)"
4,https://www.amazon.com/Apple-iPhone-XR-Fully-U...,apple,Iphone XR,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)"
...,...,...,...,...
201,https://www.amazon.com/Samsung-Galaxy-SM-G9650...,samsung,Galaxy S9 Plus,"Samsung Galaxy S9+, 64GB, Midnight Black - Ful..."
202,https://www.amazon.com/Samsung-Factory-Unlocke...,samsung,Galaxy S10,"Samsung Galaxy S10, 128GB, Prism Black - Unloc..."
203,https://www.amazon.com/Xiaomi-Note-11-Unlocked...,xiaomi,NOTE 11,Xiaomi Redmi Note 11 4G Volte 128GB + 4GB Fact...
204,https://www.amazon.com/Samsung-Galaxy-Unlocked...,samsung,Galaxy Note 10+,"Samsung Galaxy Note 10+, 256GB, Aura Glow - Fo..."


In [4]:
data_so_far = pd.read_csv("../../data/amazon_500+.csv", sep = ";", header = None)
data_so_far.columns = ["asin_url", "brand", "model", "link", "product_title",
             "name", "rating", "review_title", "country", "date",
             "specs", "first_spec", "review", "helpful", "picture",
            "verified", "profile", "video"]
data_so_far
data_so_far.asin_url.unique()

array(['https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07T3TJ8F6/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=1',
       'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07P978C2R/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=1',
       'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07P978C2R/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=2',
       ...,
       'https://www.amazon.com/Apple-iPhone-Plus-64GB-Space/product-reviews/B07YYM3HFW/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=139',
       'https://www.amazon.com/Apple-iPhone-Plus-64GB-Space/product-reviews/B07YYM3HFW/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=140',
       'https://www.amazon.com/Apple-iPhone-Plus-64G

In [5]:
'='.join(data_so_far.asin_url.unique()[0].split("=")[:-1])+"="
unique_links = []
for link in data_so_far.asin_url.unique():
    unique_link = '='.join(link.split("=")[:-1])+"="
    unique_links.append(unique_link)
unique_links

['https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07T3TJ8F6/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=',
 'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07P978C2R/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=',
 'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07P978C2R/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=',
 'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B08BGD4G36/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=',
 'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B08BGD4G36/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=',
 'https://www.amazon.com/Apple-iPhone-XR-Full

In [28]:
product_infos.iloc[197
                   ,]['link']

'https://www.amazon.com/Apple-iPhone-Plus-64GB-Space/product-reviews/B07YYM3HFW/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber='

In [8]:
all_reviews = []
missed_asins = []
num_row = 1
for index, row in product_infos[197:199].iterrows():
    print(f"{num_row}/{len(product_infos)}")
    num_row += 1
    #if row['link'] not in unique_links:
    if row['link'] == "https://www.amazon.com/Apple-iPhone-Plus-64GB-Space/product-reviews/B07YYM3HFW/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=":
        product_info = row
        get_reviews_from_page(product_info)

1/206
succesfully loaded Apple iPhone 8 Plus, 64GB, Space Gray - Unlocked (Renewed)


Exception: blocked

In [32]:
soup_reviews1 = get_soup("https://www.amazon.com/Apple-iPhone-T-Mobile-64GB-Renewed/product-reviews/B07R3CWXLW/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=1")

page successfully loaded in time 3: https://www.amazon.com/Apple-iPhone-T-Mobile-64GB-Renewed/product-reviews/B07R3CWXLW/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=1


In [37]:
soup_reviews.title.text == 'Amazon.com'

True

In [34]:
soup_reviews1.title

<title>Amazon.com: Customer reviews: Apple iPhone 8, US Version, 256GB, Gold - Verizon (Renewed)</title>

In [7]:
#This function retrieves all the review info from a page
def get_reviews_from_page(product_info):
    print(f"succesfully loaded {product_info['title']}")
    with open(f"../../data/amazon_500+.csv", "a", encoding = "UTF-8", newline = '') as csv_file:
        writer = csv.writer(csv_file, delimiter = ";")
        for i in range(142,501):
            soup_reviews = get_soup(product_info['link']+str(i))
            reviews = soup_reviews.find_all('div', {"data-hook":"review"})
            if len(reviews) == 0:
                print(f'Could not find any reviews at page {i}')
                break
            else:
                pass
            for review in reviews:
                review_info = {}
                review_info = {
                'product_url' : product_info['link']+str(i),
                'brand' : product_info['brand'],
                'model' : product_info['model'],
                'link' : product_info['link']+str(i),
                'product_title' : product_info['title']
                }
                try:
                    review_info['name'] = review.find('span', class_="a-profile-name").text
                except:
                    review_info['name'] = None
                try:
                    review_info['star_rating'] = float(review.find('i', {"data-hook":"review-star-rating"}).text.replace(' out of 5 stars',''))
                except:
                    #foreign countries
                    try:
                        review_info['star_rating'] = float(review.find('i', {"data-hook":"cmps-review-star-rating"}).text.replace(' out of 5 stars',''))
                    except:
                        review_info['star_rating'] = None
                try:
                    review_info['title'] = review.find('a', {"data-hook":"review-title"}).text.strip()
                except:
                    try:
                        #foreign countries
                        review_info['title'] = review.find('span',{"data-hook":"review-title"}).text.strip()
                    except:
                        review_info['title'] = None
                try:
                    review_info['country'] = review.find('span', {"data-hook":"review-date"}).text.replace('Reviewed in ','').rpartition(' on')[0]
                except:
                    review_info['country'] = None
                try:
                    review_info['date'] = review.find('span', {"data-hook":"review-date"}).text.replace('Reviewed in ','').rpartition(' on')[2].strip()
                except:
                    review_info['date'] = None
                attr = []
                try:
                    attributes = str(reviews[0].find('a', {"data-hook":"format-strip"}))
                    for item in re.findall(r'[>](.+?)[<]', attributes):
                        attr.append(item.replace('</i>',''))
                    review_info['info'] = attr
                except:
                    review_info['info'] = None
                try:
                    for at in attr:
                        if at.find("Color:") != -1:
                            color = at.replace("Color: ", "")
                    review_info['color'] = color
                except:
                    review_info['color'] = None
                try:
                    review_info['review'] = review.find('span', {"data-hook":"review-body"}).text.strip()
                except:
                    review_info['review'] = None
                try:
                    helpful = review.find('span', {"data-hook" : "helpful-vote-statement"}).text.strip()
                    if helpful == "One person found this helpful":
                        review_info['helpful'] = 1
                    elif "found this helpful" in helpful:
                        review_info['helpful'] = helpful.split()[0]
                    else:
                        review_info['helpful'] = helpful
                except:
                    review_info['helpful'] = None
                try:
                    if review.find('div', class_="review-image-tile-section") is None:
                        review_info['picture'] = 'no'
                    else:
                        review_info['picture'] = len(review.find('div', class_="review-image-tile-section").find_all('span'))
                except:
                    review_info['picture'] = None

                try:
                    if review.find('span', {"data-hook":"avp-badge"}) is None:
                        review_info['verified'] = 'no'
                    else:
                        review_info['verified'] = 'yes'
                except:
                    review_info['verified'] = None
                    
                try:
                    picture = review.find('div', class_="a-profile-avatar").find('noscript').text
                    if picture.find('default') != -1:
                        review_info['profile_picture'] = 'no'
                    else:
                        review_info['profile_picture'] = 'yes'
                except:
                    review_info['profile_picture'] = None
                    
                try:
                    video = review.find('div', class_="vse-airy-container vse-player-container")
                    if video == None:
                        review_info['video'] = 'no'
                    else:
                        review_info['video'] = 'yes'
                except:
                    review_info['video'] = None

                all_reviews.append(review_info)
                missing_values = [key for key,value in review_info.items() if value == None]
                if len(missing_values) > 0:
                    print(f"the following items were missing: {missing_values}")
                writer.writerow(list(review_info.values()))
            print(len(all_reviews))
            if not soup_reviews.find('li', class_ = "a-disabled a-last"):
                pass
            else:
                break

In [7]:
#to see which SKUs need to be scraped, first load in the existing dataset
amazon_data = pd.read_csv("../../gen/input/amazon.csv", sep = ";", header = None, usecols=range(16))
amazon_data.columns = ["asin_url", "brand", "model", "link", "product_title",
             "name", "rating", "review_title", "country", "date",
             "specs", "first_spec", "review", "helpful", "picture",
            "verified"]
amazon_data['asin'] = amazon_data['asin_url'].str.replace("https://www.amazon.com/dp/", "")

C:\Users\janva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
#Merge this with keepa to get the parent asin for each sku. We only want the parent and not several children since they have the same reviews
keepa_data = pd.read_excel("../../data/KeepaExport_all_variants.xlsx")
keepa_data = keepa_data[['ASIN', 'Parent ASIN']]
keepa_data.columns = ['asin', 'parent_asin']
keepa_data.parent_asin.fillna(keepa_data.asin, inplace=True)
keepa_data

,asin,parent_asin
0,B0B63XWPWF,B0B6PV17MC
1,B0B3Q23BT4,B0B6PV17MC
2,B0B3PSRHHN,B0B6PV17MC
3,B0B63WLK7Q,B0B6PV17MC
4,B0B63XWPWK,B0B6PV17MC
...,...,...
9995,B07DZNK5BS,B06Y27PBNV
9996,B09LNT1F3F,B09LP9P1FM
9997,B00KHBZDMW,B00KHBZDMW
9998,B08PP4TRMH,B09KW9M48M


In [9]:
#Merge keepa data with amazon data to see which parents were already scraped
asins_scraped = pd.merge(amazon_data, keepa_data, on = 'asin', how = 'left')
asins_scraped = asins_scraped['parent_asin'].unique()
asins_scraped

array(['B097M74GSR', 'B0B6PV17MC', 'B09JJCXC2M', 'B086RM3DQS',
       'B08L97BW9J', 'B09J6ZXBWW', 'B09J3G4TVZ', 'B0BGZ4P8JT',
       'B09HPPL95L', 'B08P7FLGL8', 'B09HFW8GRB', 'B09J6WKFL6',
       'B08ZM81H1H', 'B08W5FVK9N', nan, 'B09WR4GL4W', 'B0BGZ5NS1H',
       'B0B14LBNL6', 'B088FK1TWC', 'B088FJF1CJ', 'B097MBC1P8',
       'B09B79YT3S', 'B09WR4GL4M', 'B09VZP5RFV', 'B0B8SK885H',
       'B09T2JFWKR', 'B0B11L6YNN', 'B07YFFL2HV', 'B08HK13HGH',
       'B09Z16XW2J', 'B08H85VZNX', 'B08TVV2NCV', 'B08L6QVY85',
       'B0BGZ9P4P9', 'B098BK2SJ5', 'B08KRKFHGV', 'B097M71WR1',
       'B08QFZRVKV', 'B098KF1G4Q', 'B09V1M7KTM', 'B084KS5WYC',
       'B076M9RQJ2', 'B09T1ZMF1X', 'B09HGHDV9L', 'B08H7RJLM2',
       'B0B3C1PTNG', 'B07758RMKD', 'B085P16MD8', 'B08ZWJXK1Q',
       'B07P1BKNHP', 'B0B82GC3D5', 'B07YD6MT5N', 'B09GBLFSB2',
       'B09QNWPHX3', 'B09G2CXLMX', 'B096T6PF3G', 'B09BTB5MKT',
       'B086H4C8CX', 'B081TLRQ9H', 'B08ZHNGKHB', 'B09Q7VXHFH',
       'B09JT5P2BG', 'B08H8DH3RM', 'B08FRT6G9W', '

In [10]:
#Get the links of the SKUs
amazon_links_category = pd.read_csv("../../gen/input/amazon_links_from_category_0_100.csv", sep = ";")
amazon_links_category

,Unnamed: 0,title_x,asin,link,title_y,reviews,parent_asin,brand,color
0,0,Google-Pixel-6a-Smartphone-Megapixel,B0B3PSRHHN,https://www.amazon.com/Google-Pixel-6a-Smartph...,Google Pixel 6a - 5G Android Phone - Unlocked ...,250,B0B6PV17MC,Google,Charcoal
1,1,Google-Pixel-6a-Smartphone-Megapixel,B0B3PQPNKS,https://www.amazon.com/Google-Pixel-6a-Smartph...,Google Pixel 6a - 5G Android Phone - Unlocked ...,150,B0B6PV17MC,Google,Chalk
2,3,Apple-iPhone-11-64GB-Green,B081Y5H8P4,https://www.amazon.com/Apple-iPhone-11-64GB-Gr...,"Apple iPhone 11, US Version, 64GB, Green - AT&...",27500,B097M74GSR,Apple,Green
3,4,Samsung-Galaxy-128GB-Prism-Black,B082T4F34B,https://www.amazon.com/Samsung-Galaxy-128GB-Pr...,"Samsung Galaxy S10e, 128GB, Prism Black - Unlo...",9550,B09JJCXC2M,SAMSUNG,Prism Black
4,5,Samsung-Galaxy-Cardinal-Verizon-Renewed,B08D34KDKJ,https://www.amazon.com/Samsung-Galaxy-Cardinal...,"Samsung Galaxy S10e, 128GB, Cardinal Red - Ver...",9500,B09JJCXC2M,SAMSUNG,Cardinal Red
...,...,...,...,...,...,...,...,...,...
1896,2919,Nokia-Dual-Sim-Factory-Unlocked-Smartphone,B0BF14B54G,https://www.amazon.com/Nokia-Dual-Sim-Factory-...,NaN,0,NaN,NaN,NaN
1897,2920,Fairphone-Dual-SIM-Factory-Unlocked-Smartphone,B09HHPD4WM,https://www.amazon.com/Fairphone-Dual-SIM-Fact...,Fairphone 4 Dual-SIM 128GB ROM + 6GB RAM (GSM ...,0,B09HHPD4WM,Fairphone,Grey
1898,2921,Motorola-One-Zoom-Unlocked-T-Mobile,B07W6KTDMD,https://www.amazon.com/Motorola-One-Zoom-Unloc...,Motorola One Zoom | Unlocked | GSM only | 4/12...,50,B07W6KTDMD,Motorola,Bronze
1899,2923,BlackBerry-BBE100-2-Unlocked-Phone-Camera,B07M65HTRY,https://www.amazon.com/BlackBerry-BBE100-2-Unl...,BlackBerry KEY2 LE 64GB - GSM Unlocked Android...,0,B07MCKZMYN,BlackBerry,Dark Blue/Gold


In [11]:
all_reviews = []
missed_asins = []
i = 1
for index,row in amazon_links_category[50:500].iterrows():
    #Scrape only parent asins that are not already in the dataset
    if row['parent_asin'] not in asins_scraped:
        print(f"{round(i/len(amazon_links_category[50:500])*100)}%")
        i += 1
        product_info = {
        'link' : row['link'],
        }
        product_info['brand'] = row['brand']
        #No info about model on the category page, so none for now
        product_info['model'] = None
        product_info['title'] = row['title_y']
        get_reviews_from_page(product_info)

0%
succesfully loaded Samsung Galaxy S9 G960U 64GB - Unlocked Blue (Renewed)
page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-S9-G960U-64GB/product-reviews/B09K4GPHTL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
10
page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-S9-G960U-64GB/product-reviews/B09K4GPHTL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
20
page successfully loaded in time 

page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-S9-G960U-64GB/product-reviews/B09K4GPHTL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=18
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
179
0%
succesfully loaded Samsung Galaxy S10E 128GB 5.8" 4G LTE Fully Unlocked, Black (Renewed)
page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-S10E-Fully-Unlocked-Renewed/product-reviews/B09DRHTSYZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
the following items were missing: ['color', 'helpful']
the following items were missing: ['color']
the following items were missing: ['color']
the following items were missing: ['color', 'h

page successfully loaded in time 3: https://www.amazon.com/SAMSUNG-Galaxy-5G-Factory-Unlocked/product-reviews/B08HPTQRJS/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=9
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
313
page successfully loaded in time 3: https://www.amazon.com/SAMSUNG-Galaxy-5G-Factory-Unlocked/product-reviews/B08HPTQRJS/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=10
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
323
page successfully loaded in time 3: https://www.amazon.com/SAMSUNG-Galaxy-5G-Factory-Unlocked/p

page successfully loaded in time 3: https://www.amazon.com/Battery-Unlocked-Motorola-Camera-Renewed/product-reviews/B09X3YNV4N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
Could not find any reviews at page 1
2%
succesfully loaded Motorola Edge | 2022 | 2-Day Battery | Unlocked | Made for US by Motorola | 8/256GB | 50MP Camera | Mineral Gray
page successfully loaded in time 3: https://www.amazon.com/Motorola-Battery-Unlocked-Camera-Mineral/product-reviews/B0B7XYTXXL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
the following items were missing: ['color']
the following items were missing: ['color']
the following items were missing: ['color']
the following items were missing: ['color']
the following items were missing: ['color']
the following items were missing: ['color']
460
page successfully loaded in time 3: https://www.amazon.com/Motorola-Battery-Unlocked-Camera-Mineral/product-reviews/B0B7XYTXXL/ref=cm_cr_dp_d_show_all_btm?ie=

page successfully loaded in time 3: https://www.amazon.com/Samsung-SM-A525M-International-Version-Warranty/product-reviews/B08ZG86B6S/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=18
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
640
page successfully loaded in time 3: https://www.amazon.com/Samsung-SM-A525M-International-Version-Warranty/product-reviews/B08ZG86B6S/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=19
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were m

page successfully loaded in time 3: https://www.amazon.com/Unlocked-Smartphone-6-26in-Screen-Android/product-reviews/B08RZ4D19Y/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
the following items were missing: ['color']
the following items were missing: ['color']
the following items were missing: ['color']
the following items were missing: ['color']
the following items were missing: ['color', 'helpful']
the following items were missing: ['color', 'helpful']
the following items were missing: ['color']
the following items were missing: ['color']
the following items were missing: ['color']
the following items were missing: ['color']
761
page successfully loaded in time 3: https://www.amazon.com/Unlocked-Smartphone-6-26in-Screen-Android/product-reviews/B08RZ4D19Y/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
the following items were missing: ['color', 'helpful']
the following items were missing: ['color']
the following items were missin

page successfully loaded in time 3: https://www.amazon.com/Smallest-Mobile-Phone-L8Star-Unlocked/product-reviews/B097XTKYST/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=11
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
921
4%
succesfully loaded Motorola Edge + |2022| 4800mAh Battery | Unlocked | Made for US by Motorola | 8/512GB | 50MP Camera | Cosmos Blue
page successfully loaded in time 3: https://www.amazon.com/Motorola-4800mAh-Battery-Unlocked-Camera/product-reviews/B09TG89P52/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']


page successfully loaded in time 3: https://www.amazon.com/Smartphone-Android-Unlocked-Children-Cellphone/product-reviews/B09M9XK4H9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
the following items were missing: ['helpful']
the following items were missing: ['helpful']
1015
page successfully loaded in time 3: https://www.amazon.com/Smartphone-Android-Unlocked-Children-Cellphone/product-reviews/B09M9XK4H9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
Could not find any reviews at page 2
5%
succesfully loaded Samsung Galaxy A13 (SM-A135M/DS) Dual SIM,64 GB 4GB RAM, Factory Unlocked GSM, International Version - No Warranty - (White)
page successfully loaded in time 3: https://www.amazon.com/Samsung-SM-A135M-DS-Unlocked-International/product-reviews/B09X63LMV2/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
the following items were missing: ['color', 'helpful']
1016
page successfully loaded in time 3: https:

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))